## Setup

In [ ]:
!git clone https://github.com/huggingface/diffusers
%pip install git+https://github.com/huggingface/diffusers
%pip install triton
%pip install accelerate transformers ftfy gradio natsort safetensors
%pip install bitsandbytes
!pip install torch
!pip install PIL
!pip install xformers
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux
#!pip install requests
!pip install git+https://github.com/huggingface/accelerate.git
!wget -O civitai_midel.ckpt https://civitai.com/api/download/models/201259?type=Model&format=SafeTensor&size=pruned&fp=fp16
!wget https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml

In [ ]:
import os
import requests
from shutil import copyfileobj
# from google.colab import files, drive
from PIL import Image
import cv2
import numpy as np

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler, StableDiffusionControlNetPipeline, UniPCMultistepScheduler
from diffusers.utils import load_image
from IPython.display import display

In [ ]:
!pip list | grep torch

In [ ]:
!python diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py \
      --checkpoint_path='/kaggle/input/cvtai/other/1/1/epicphotogasm_zUniversal.safetensors'\
      --original_config_file='/kaggle/working/v1-inference.yaml'\
      --dump_path='/kaggle/working/cache_dir/models/civitai_model'\
      --scheduler_type="ddim" --prediction_type='epsilon'\
      --from_safetensors

In [ ]:
# drive.mount('/content/drive')
# !cp -r /content/cache_dir/models/civitai_model/ /content/drive/MyDrive/

''' url = 'https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py'
r = requests.get(url)
with open('train_dreambooth_lora.py', 'w') as f:
    f.write(r.text)
#import train_dreambooth_lora as tl '''

## Data

In [ ]:
if not os.path.exists('/kaggle/working/data'):
    os.makedirs('/kaggle/working/data')

def download_image(url, destination):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(destination, 'wb') as file:
            copyfileobj(response.raw, file)
    else:
        print(f"Failed to download image from {url}")

In [ ]:
if not os.path.exists('/kaggle/working/output_sd'):
    os.makedirs('/kaggle/working/output_sd')

In [ ]:
urls = ['https://th.bing.com/th/id/OIP.2hBjcfycMBrbaPnL3wXu8gHaE8?rs=1&pid=ImgDetMain', 'https://th.bing.com/th/id/R.3e2bd135692331de02d86b52fcf7b275?rik=O7lPjYvnwsODwA&pid=ImgRaw&r=0', 'https://th.bing.com/th/id/R.8e1bc3aae2d737df41d876b272382eaa?rik=dKXjcqMd8KgCcQ&riu=http%3a%2f%2fwww.trussvilletribune.com%2fwp-content%2fuploads%2f2017%2f09%2fct-donald-trump-stupid-president-20170531.jpg&ehk=OhcPiDeTW%2bwYaA2bmKh1PnAX8UDY7y93P6ASzSrH%2bTE%3d&risl=&pid=ImgRaw&r=0',
        'https://static01.nyt.com/images/2019/09/10/opinion/10bruniWeb/10bruniWeb-superJumbo.jpg?quality=90&auto=webp', 'https://th.bing.com/th/id/OIP.UfrbIgUcA723drH6oIsVkAAAAA?rs=1&pid=ImgDetMain', 'https://i0.wp.com/us-east-1.linodeobjects.com/gunaxin/2019/04/Trump.jpg?fit=1291,1291&ssl=1',
        'https://mediaproxy.salon.com/width/1200/https://media.salon.com/2020/11/donald-trump-1123202.jpg', 'https://d.newsweek.com/en/full/1289369/president-donald-trump.jpg', 'https://th.bing.com/th/id/OIP.JxymTxgdNKMN7zf8gpLquwAAAA?rs=1&pid=ImgDetMain',
        'https://th.bing.com/th/id/R.8ae357a671fdede9aae087afcdaef8ed?rik=zJPKrpeWjV%2fBLA&riu=http%3a%2f%2fwp.production.patheos.com%2fblogs%2fjappersandjanglers%2ffiles%2f2016%2f02%2fDonald_Trump_August_2015.jpg&ehk=kRGzsWqJUHd7a1MRaU0Ujtf%2bct2H0WzI4L8iBk5PDgo%3d&risl=1&pid=ImgRaw&r=0', 'https://static.standard.co.uk/s3fs-public/thumbnails/image/2019/09/30/11/DonaldTrump3009a.jpg', 'https://th.bing.com/th/id/R.bd0d97016599feac2964e758f696116a?rik=ar%2bgIVkQ3dUaGg&riu=http%3a%2f%2fmedia.al.com%2fnews_impact%2fphoto%2fdonald-trump-684c376f6a658f7c.jpg&ehk=%2fk9v46JmmABW1GqUtDyNfODrHfdy1tjzfIBtNGBciJw%3d&risl=&pid=ImgRaw&r=0',
        'https://th.bing.com/th/id/R.bbd7208dd5a2ad77468829e14b135523?rik=QsZoZ%2f4tL6Ywbw&riu=http%3a%2f%2fmedia.advance.net%2fpolitics_news_national_desk%2fphoto%2fdonald-trump-b19c8bfd95bf47df.jpg&ehk=RwrCtWiKGFmqNidq4J1Oe7ZxSZG6Tfcam%2bc3UT4HAJI%3d&risl=&pid=ImgRaw&r=0', 'https://mediaproxy.salon.com/width/1200/https://media.salon.com/2016/05/donald_trump132.jpg', 'https://thesunflower.com/wp-content/uploads/2016/11/Trump45th.jpg',
        'https://i0.wp.com/thexplorion.com/wp-content/uploads/2020/09/Donald-Trump-scaled-1.jpg?w=2560&ssl=1', 'https://th.bing.com/th/id/R.fa723c6c99a985ecea8952644358d43a?rik=jnr%2fHgyQtX7xAQ&riu=http%3a%2f%2fupload.wikimedia.org%2fwikipedia%2fcommons%2fe%2fee%2fDonald_Trump_by_Gage_Skidmore.jpg&ehk=3SX1oikep4EofGgeIxaSxhxn5jCr3G0rbN6qeMT%2bmZM%3d&risl=1&pid=ImgRaw&r=0', 'https://th.bing.com/th/id/OIP.5tI0yA0QfTUnR6bVrC2tKAHaE7?rs=1&pid=ImgDetMain',
        'https://static.billboard.com/files/media/02-donald-trump-jan-press-conference-2017-billboard-1548-compressed.jpg', 'https://th.bing.com/th/id/OIP.EpmdjXj44XAVCfyLAOxehQAAAA?rs=1&pid=ImgDetMain', 'https://www.thefamouspeople.com/profiles/images/donald-trump-10.jpg',
        'https://www.gannett-cdn.com/presto/2019/02/13/USAT/6b7a8c4b-11ed-416a-9c8a-c8eeca76441a-EPA_USA_TRUMP_WHITE_HOUSE_2.JPG?crop=5183,2915,x1,y259&width=3200&height=1680&fit=bounds', 'https://www.sott.net/image/s28/573063/full/president_donald_trump.jpg', 'https://th.bing.com/th/id/OIP.8Idf1Roilu2HkpPuiTcdMgAAAA?rs=1&pid=ImgDetMain',
        'https://th.bing.com/th/id/OIP.kLDNE0qwRbBTM7QWj9j4FAAAAA?rs=1&pid=ImgDetMain', 'https://th.bing.com/th/id/R.3b90b49abb7d9c3c63ff59cae2a9f3b6?rik=LIhSRxyZNZi%2fYg&riu=http%3a%2f%2fcbsnews1.cbsistatic.com%2fhub%2fi%2f2016%2f12%2f27%2f0bdd3d20-ff41-45ef-bc71-7711a3d22540%2ftrump.jpg&ehk=WdB9LpOlukT7ceXiiyxJh0cHzYe5%2fUrFFGeJk9EeSTM%3d&risl=&pid=ImgRaw&r=0', 'https://static.politico.com/e9/06/031ab3b1406fb076ba8866a93748/150904-donald-trump-2-gty-629.jpg',
        'https://img.washingtonpost.com/rw/2010-2019/WashingtonPost/2016/02/01/Editorial-Opinion/Images/250663066_4.jpg?uuid=WGqwrskyEeWnslovgksCyQ', 'https://d.newsweek.com/en/full/485398/donald-trump.jpg', 'https://static01.nyt.com/images/2016/10/25/us/25TRUMPTAPES1/25TRUMPTAPES1-superJumbo.jpg',
        'https://d.newsweek.com/en/full/635470/donald-trump.jpg', 'https://img.etimg.com/photo/msid-83054761/Donald%20Trump.jpg', 'https://th.bing.com/th/id/OIP.eLn7RWUCqJi7S_3lMdu-4wAAAA?rs=1&pid=ImgDetMain',
        'https://cloudfront-us-east-2.images.arcpublishing.com/reuters/M3WJEQDT75MHTC6OWAOB7DPAI4.jpg', 'https://th.bing.com/th/id/OIP.-gsuwTx5STfjlyGHg9_FcwHaE3?w=640&h=420&rs=1&pid=ImgDetMain', 'https://cdn.cnn.com/cnnnext/dam/assets/161128072443-01-trump-1128-super-tease.jpg',
        'https://www.thenation.com/wp-content/uploads/2016/10/Trump_Donald_Debate_ap_img.jpg', 'https://th.bing.com/th/id/OIP.7IrIy2EAhn03waUAbInCBAHaE8?rs=1&pid=ImgDetMain', 'https://mediaproxy.salon.com/width/960/https://media.salon.com/2017/08/donald-trump60.jpg',
        'https://th.bing.com/th/id/OIP.4wcUv4t8WIkLnSm5gLnEsgHaE8?rs=1&pid=ImgDetMain', 'https://static.independent.co.uk/s3fs-public/thumbnails/image/2017/11/23/15/donald-trump.jpg', 'https://mediaproxy.salon.com/width/1200/https://media.salon.com/2016/04/donald_trump129.jpg',
        'https://www.thenation.com/wp-content/uploads/2017/04/Donald_Trump_CC_img.jpg?scale=896&compress=80', 'https://d.newsweek.com/en/full/694860/rtx341qy.jpg', 'https://www.michaeltyler.co.uk/wp-content/uploads/2016/03/donald-trump-e1461655495762.jpg']

for enum, url in enumerate(urls):
    destination = os.path.join('/kaggle/working', f'pic_{enum}.jpg')
    download_image(url, destination)

In [ ]:
dataroot = "/kaggle/input/trump-brime-trimmed"

## Stable diffusion 1.5

In [ ]:
if not os.path.exists('/kaggle/working/class_data_dir'):
    os.makedirs('/kaggle/working/class_data_dir')

os.environ['INSTANCE_DIR'] = "/kaggle/input/trump-brime-trimmed"
os.environ['CLASS_DIR'] = "/kaggle/working/class_data_dir"

os.environ['MODEL_NAME'] = "/kaggle/working/cache_dir/models/civitai_model/"
os.environ['OUTPUT_DIR'] = "/kaggle/working/output_sd"

In [ ]:
!python3 /kaggle/working/diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --class_data_dir=$CLASS_DIR \
  --instance_prompt="a photo of sks man face" \
  --class_prompt="a photo of man face " \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --gradient_accumulation_steps=1 \
  --num_class_images=500 \
  --max_train_steps=800 \
  --checkpointing_steps=800 \
  --use_8bit_adam \
  --mixed_precision="no"\
  --train_text_encoder

In [ ]:
model_path = os.environ['OUTPUT_DIR']

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
g_cuda=None

In [ ]:
g_cuda = torch.Generator(device='cuda')
seed = 345252
g_cuda.manual_seed(seed)

In [ ]:
prompt = "portrait of sks man face, on the street, lights, midnight, NY, standing, 4K, raw, hrd, hd, high quality, realism, sharp focus,  sarcastic eyes, detailed eyes, ironic expression"
negative_prompt = "deformed, distorted, disfigured, poorly drawn, bad anatomy, extra limb, missing limb, floating limbs, mutated hands disconnected limbs, mutation, ugly, blurry, amputation"
num_samples = 2
guidance_scale = 7.5
num_inference_steps = 35
height = 768
width = 1024

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)

In [ ]:
token = "sks"
promt_list = [
    {
     "name": "kitchen",
     "prompt":f"close up portrait of {token} man face, in the kitchen, standing, 4K, raw, hrd, hd, high quality, realism, sharp focus",
     "n_prompt":"naked, nsfw, deformed, distorted, disfigured, poorly drawn, bad anatomy, extra limb, missing limb, floating limbs, mutated hands disconnected limbs, mutation, ugly, blurry, amputation",
    },
    {
     "name": "forest",
     "prompt":f"portrait of {token} man face, in the forest, standing, 4K, raw, hrd, hd, high quality, realism, sharp focus",
     "n_prompt":"naked, nsfw, deformed, distorted, disfigured, poorly drawn, bad anatomy, extra limb, missing limb, floating limbs, mutated hands disconnected limbs, mutation, ugly, blurry, amputation",
    },
    {
     "name": "street",
     "prompt":f"portrait of {token} man face, on the street, lights, midnight, NY, standing, 4K, raw, hrd, hd, high quality, realism, sharp focus,  beautiful eyes, detailed eyes",
     "n_prompt":"naked, nsfw, deformed, distorted, disfigured, poorly drawn, bad anatomy, extra limb, missing limb, floating limbs, mutated hands, mutation, ugly, blurry",
    },
    {
     "name": "anime",
     "prompt":f"portrait of {token} man face, anime style, japan, midnight, Tokyo, standing, 4K, raw, hrd, hd, high quality, sharp focus, detailed eyes",
     "n_prompt":"naked, nsfw, deformed, distorted, disfigured, poorly drawn, bad anatomy, extra limb, missing limb, floating limbs, mutated hands, mutation, ugly, blurry",
    },
    {
     "name": "classicist art",
     "prompt":f"portrait of {token} man face, classic art style, medieval europe, moonlight, standing, 4K, raw, hrd, hd, high quality, sharp focus, detailed eyes",
     "n_prompt":"naked, nsfw, deformed, distorted, disfigured, poorly drawn, bad anatomy, extra limb, missing limb, floating limbs, mutated hands, mutation, ugly, blurry",
    },
]

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [25]:
seed = 1666
repeat = 2
num_samples = 2
guidance_scale = 7.5
num_inference_steps = 30
height = 768
width = 1024


if not os.path.exists('/kaggle/working/output_pics_sd'):
    os.makedirs('/kaggle/working/output_pics_sd')

save_folder = "/kaggle/working/output_pics_sd"
save_mode = "with_train_token"
# save_mode = "base_model"

In [26]:
for idx, sample in enumerate(promt_list):
    prompt = sample.get("prompt")
    negative_prompt = sample.get("n_prompt")
    name = sample.get("name")
    image_list = []
    for _ in range(repeat):
        generator = g_cuda
        with autocast("cuda"), torch.inference_mode():
            images = pipe(
                prompt,
                height=height,
                width=width,
                negative_prompt=negative_prompt,
                num_images_per_prompt=num_samples,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                generator=generator
            ).images
        image_list.extend(images)
        seed+=345324
    img_grid = image_grid(image_list, num_samples, repeat)
    save_path = os.path.join(save_folder, save_mode, f"{height}x{width}")
    os.makedirs(save_path, exist_ok=True)
    img_grid.save(os.path.join(save_path, f"{name}.jpg"))

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
%pip install peft

In [ ]:
if not os.path.exists('/kaggle/working/class_data_lora'):
    os.makedirs('/kaggle/working/class_data_lora')

if not os.path.exists('/kaggle/working/output_lora_1'):
    os.makedirs('/kaggle/working/output_lora_1')
if not os.path.exists('/kaggle/working/output_lora_2'):
    os.makedirs('/kaggle/working/output_lora_2')
if not os.path.exists('/kaggle/working/output_lora_3'):
    os.makedirs('/kaggle/working/output_lora_3')

# os.environ['INSTANCE_DIR'] = "/kaggle/working/data_cr"
os.environ['CLASS_DIR_LORA'] = "/kaggle/working/class_data_lora"

# os.environ['MODEL_NAME'] = "/kaggle/working/cache_dir/models/civitai_model/"
os.environ['OUTPUT_DIR_LORA_1'] = "/kaggle/working/output_lora_1"
os.environ['OUTPUT_DIR_LORA_2'] = "/kaggle/working/output_lora_2"
os.environ['OUTPUT_DIR_LORA_3'] = "/kaggle/working/output_lora_3"

In [ ]:
!python3 /kaggle/working/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR_LORA_1 \
  --instance_prompt="a photo of sks man face" \
  --class_prompt="a photo of man face " \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1111 \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=5e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --gradient_accumulation_steps=1 \
  --num_class_images=500 \
  --max_train_steps=800 \
  --checkpointing_steps=800 \
  --use_8bit_adam \
  --mixed_precision="no"\
  --train_text_encoder \
  --rank=4

In [ ]:
!python3 /kaggle/working/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR_LORA_2 \
  --instance_prompt="a photo of sks man face" \
  --class_prompt="a photo of man face " \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1111 \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=5e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --gradient_accumulation_steps=1 \
  --num_class_images=500 \
  --max_train_steps=800 \
  --checkpointing_steps=800 \
  --use_8bit_adam \
  --mixed_precision="no"\
  --train_text_encoder \
  --rank=8

In [ ]:
!python3 /kaggle/working/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR_LORA_3 \
  --instance_prompt="a photo of sks man face" \
  --class_prompt="a photo of man face " \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1111 \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=5e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --gradient_accumulation_steps=1 \
  --num_class_images=500 \
  --max_train_steps=800 \
  --checkpointing_steps=800 \
  --use_8bit_adam \
  --mixed_precision="no"\
  --train_text_encoder \
  --rank=24

In [ ]:
def lora_infer(model_path, weights, save_folder):
    pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    pipe.load_lora_weights(weights)
    g_cuda=None
    
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    save_folder = save_folder
    save_mode = "base_model"
    for idx, sample in enumerate(promt_list):
    prompt = sample.get("prompt")
    negative_prompt = sample.get("n_prompt")
    name = sample.get("name")
    image_list = []
    for _ in range(repeat):
        generator = torch.Generator("cuda").manual_seed(seed)
        with autocast("cuda"), torch.inference_mode():
            images = pipe(
                prompt,
                height=height,
                width=width,
                negative_prompt=negative_prompt,
                num_images_per_prompt=num_samples,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                generator=generator
            ).images
        image_list.extend(images)
        seed+=345324

In [ ]:
model_path1 = os.environ['OUTPUT_DIR_LORA_1']
model_path2 = os.environ['OUTPUT_DIR_LORA_2']
model_path3 = os.environ['OUTPUT_DIR_LORA_3']

weights1 = "/kaggle/working/output_lora_1/diffusion_pytorch_lora.bin"
weights2 = "/kaggle/working/output_lora_2/diffusion_pytorch_lora.bin"
weights3 = "/kaggle/working/output_lora_3/diffusion_pytorch_lora.bin"

save_folder1 = '/content/output_pics_lora_rank_4'
save_folder2 = '/content/output_pics_lora_rank_8'
save_folder3 = '/content/output_pics_lora_rank_24'



lora_infer(model_path1, weights1, save_folder1)
lora_infer(model_path2, weights2, save_folder2)
lora_infer(model_path3, weights3, save_folder3)

## Unet vs. Lora comparison

## ControlNet